<a href="https://colab.research.google.com/github/svondracek0/sda_contest/blob/main/regrese_lekce_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Supervised learning - Regresni problemy

In [ ]:
# import knihoven
import pandas as pd
import numpy as np
from plotly import express
import seaborn as sns

# Rozpis kroků:
* I. Načtení dat, základní popisné statistiky, základní EDA
* II. Základní feature engineering
* III. Trénování modelů
* IV. Vyhodnocení modelů na validačních datech

## 1 Načtení dat

In [ ]:
# nacteme data pochazejici z kaggle.com: https://www.kaggle.com/datasets/sidharth178/car-prices-dataset
# data je nutne nahrat do colab prostredi "drag&drop"

df = pd.read_csv("./train.csv", index_col=0)
df_valid = pd.read_csv("./test.csv", index_col = 0)

In [ ]:
# sloupce tabulky
df.columns

Index(['Price', 'Levy', 'Manufacturer', 'Model', 'Prod. year', 'Category',
       'Leather interior', 'Fuel type', 'Engine volume', 'Mileage',
       'Cylinders', 'Gear box type', 'Drive wheels', 'Doors', 'Wheel', 'Color',
       'Airbags'],
      dtype='object')

In [ ]:
# nektere sloupce obsahuji mezery, velka pismena - lepsi sjednotit
df.columns = [col.lower().replace(" ", "_").replace(".", "") for col in df.columns]
df_valid.columns = [col.lower().replace(" ", "_").replace(".", "") for col in df_valid.columns]

In [ ]:
# mnozstvi jednotlivych vyrobcu
df['manufacturer'].value_counts()

,count
manufacturer,
HYUNDAI,3769
TOYOTA,3662
MERCEDES-BENZ,2076
FORD,1111
CHEVROLET,1069
...,...
TESLA,1
PONTIAC,1
SATURN,1


In [ ]:
# datove type - potrebujeme prevest object na ciselne hodnoty
df.dtypes

,0
price,int64
levy,object
manufacturer,object
model,object
prod_year,int64
category,object
leather_interior,object
fuel_type,object
engine_volume,object
mileage,object


## 2 Preprocessing

In [ ]:
def remove_outliers_quantile(df, col, quantile=.95, low=False, high=True):
  """
  Odstarnuje odlehlehle hodnoty pro vybray sloupcec - smaze dotcene radky
  """
  q_low = -np.inf
  q_high = np.inf
  if low:
    q_low = df[col].quantile(1-quantile)
  if high:
    q_high = df[col].quantile(quantile)
  df_removed = df[(df[col] < q_high) & (df[col] > q_low)]
  return df_removed

In [ ]:
# jednoduche upravy datasetu predtim nez s nim budeme pracovat
def preprocess(df):
  df_preprocessed = df.copy()  # kopie, abychom neupravovali puvodni tabulku
  df_preprocessed.columns = [col.lower().replace(" ", "_").replace(".", "") for col in df_preprocessed.columns]  # sjednoceni nazvu sloupcu
  df_preprocessed = df_preprocessed.drop("levy", axis=1)  # odstraneni sloupce, ktery nema znamy vyznam
  df_preprocessed['mileage'] = df_preprocessed['mileage'].str.split(" ").apply(lambda x: x[0]).astype("int")  # pretypovani na cislo, odstraneni jednotky
  df_preprocessed['doors'] = df_preprocessed['doors'].str.split("-").apply(lambda x: x[0]).str.replace("0", "")  # zjednoduseni kodovani
  df_preprocessed['turbo'] = df_preprocessed['engine_volume'].str.contains("Turbo")  # nova promenna, extrahujeme z puvodni engine_volume
  df_preprocessed['engine_volume'] = df_preprocessed['engine_volume'].str.split(" ").apply(lambda x: x[0]).astype("float")  # prevedeme engine volume na cislo
  df_preprocessed = remove_outliers_quantile(df_preprocessed, "price", high=True)  # odstraneni outliers
  df_preprocessed = remove_outliers_quantile(df_preprocessed, "mileage", high=True)  # odstraneni outliers
  return df_preprocessed

In [ ]:
preprocess(df)

,price,manufacturer,model,prod_year,category,leather_interior,fuel_type,engine_volume,mileage,cylinders,gear_box_type,drive_wheels,doors,wheel,color,airbags,turbo
ID,,,,,,,,,,,,,,,,,
45654403,13328,LEXUS,RX 450,2010,Jeep,Yes,Hybrid,3.5,186005,6.0,Automatic,4x4,4,Left wheel,Silver,12,False
44731507,16621,CHEVROLET,Equinox,2011,Jeep,No,Petrol,3.0,192000,6.0,Tiptronic,4x4,4,Left wheel,Black,8,False
45774419,8467,HONDA,FIT,2006,Hatchback,No,Petrol,1.3,200000,4.0,Variator,Front,4,Right-hand drive,Black,2,False
45769185,3607,FORD,Escape,2011,Jeep,Yes,Hybrid,2.5,168966,4.0,Automatic,4x4,4,Left wheel,White,0,False
45809263,11726,HONDA,FIT,2014,Hatchback,Yes,Petrol,1.3,91901,4.0,Automatic,Front,4,Left wheel,Silver,4,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45798355,8467,MERCEDES-BENZ,CLK 200,1999,Coupe,Yes,CNG,2.0,300000,4.0,Manual,Rear,2,Left wheel,Silver,5,True
45778856,15681,HYUNDAI,Sonata,2011,Sedan,Yes,Petrol,2.4,161600,4.0,Tiptronic,Front,4,Left wheel,Red,8,False
45804997,26108,HYUNDAI,Tucson,2010,Jeep,Yes,Diesel,2.0,116365,4.0,Automatic,Front,4,Left wheel,Grey,4,False


In [ ]:
# zvoleni vsech pouzitych promennych
selected_features = ['price', 'prod_year', 'category', 'manufacturer',
                     'leather_interior', 'fuel_type', 'engine_volume',
                     'mileage', 'cylinders', 'gear_box_type', 'drive_wheels',
                     'doors', 'wheel', 'airbags', 'turbo']

In [ ]:
# slice tabulky podle zvolenych promennych
df_preprocessed = preprocess(df).loc[:, selected_features]

## 2 EDA

In [ ]:
!pip install ydata-profiling

In [ ]:
# vyvoreni profile reportu
from ydata_profiling import ProfileReport
report = ProfileReport(df_preprocessed)

In [ ]:
# report ulozime do html
report.to_file("report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 15/15 [00:00<00:00, 18.48it/s]
/usr/local/lib/python3.11/dist-packages/ydata_profiling/model/correlations.py:87: UserWarning: There was an attempt to calculate the auto correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"auto": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/ydataai/ydata-profiling/issues
(include the error message: 'cannot reindex on an axis with duplicate labels')
  warnings.warn(


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [ ]:
# aplikujeme transformace nasich promennych
def engineer_features(df):
  df_engineered = df.copy()  # kopie,  abychom nemenili puvodni df
  df_engineered = df_engineered.loc[:, selected_features] # pouze vybrane promenne
  string_cols = df_engineered.select_dtypes("object").columns  # kategoricke promenne
  df_engineered = pd.get_dummies(df_engineered, columns=string_cols)  # kategoricke prmenne one-hot encodovane
  num_cols = [col for col in df_engineered.select_dtypes(np.number).columns if col != "price"]  # numericke promenne
  df_engineered.loc[:, num_cols] = scaler.fit_transform(df_engineered.loc[:, num_cols])  # numericke promenne standardne skalovane
  return df_engineered

In [ ]:
df_engineered = engineer_features(df_preprocessed)
df_engineered.head(5)

<ipython-input-16-c28815019f8a>:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-0.13348844  0.04630513 -0.85266274 ... -0.13348844 -0.67286917
  0.22609871]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_engineered.loc[:, num_cols] = scaler.fit_transform(df_engineered.loc[:, num_cols])  # numericke promenne standardne skalovane
<ipython-input-16-c28815019f8a>:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 0.74210256  0.81972889  0.92331698 ... -0.15963179 -1.00267054
  0.75398929]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_engineered.loc[:, num_cols] = scaler.fit_transform(df_engineered.loc[:, num_cols])  # numericke promenne standardne skalovane
<ipython-input-16-c28815019f8a>:8: FutureWarning: Setting an item of incompatible dtype is depreca

,price,prod_year,engine_volume,mileage,cylinders,airbags,turbo,category_Cabriolet,category_Coupe,category_Goods wagon,...,gear_box_type_Tiptronic,gear_box_type_Variator,drive_wheels_4x4,drive_wheels_Front,drive_wheels_Rear,doors_2,doors_4,doors_>5,wheel_Left wheel,wheel_Right-hand drive
ID,,,,,,,,,,,,,,,,,,,,,
45654403,13328,-0.133488,1.463691,0.742103,1.254395,1.250140,False,False,False,False,...,False,False,True,False,False,False,True,False,True,False
44731507,16621,0.046305,0.867348,0.819729,1.254395,0.324357,False,False,False,False,...,True,False,True,False,False,False,True,False,True,False
45774419,8467,-0.852663,-1.160216,0.923317,-0.472101,-1.064317,False,False,False,False,...,False,True,False,True,False,False,True,False,False,True
45769185,3607,0.046305,0.271006,0.521473,-0.472101,-1.527209,False,False,False,False,...,False,False,True,False,False,False,True,False,True,False
45809263,11726,0.585686,-1.160216,-0.476404,-0.472101,-0.601426,False,False,False,False,...,False,False,False,True,False,False,True,False,True,False


## 3 Modelovani

In [ ]:
# split na train a test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_engineered.drop("price",
                                                                       axis=1),
                                                    df_engineered["price"],
                                                    test_size=0.2, random_state=42)

In [ ]:
# dodatecne knihovny pro gradient boosting algoritmy
!pip install xgboost
!pip install lightgbm
!pip install catboost

In [ ]:
# import knihoven pro modelovani
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [ ]:
# vybirame pouze modely kteres se ukazaly ze funguji, muzete odkomentovat a nechat sebenout
grid_dict = {
    "LinearRegression": LinearRegression(),
    # "Ridge": Ridge(),
    # "Lasso": Lasso(),
    # "SVR": SVR(),
    # "DecisionTreeRegressor": DecisionTreeRegressor(),
    # "RandomForestRegressor": RandomForestRegressor(),
    # "GradientBoostingRegressor": GradientBoostingRegressor(),
    "XGBRegressor": XGBRegressor(verbosity=2),
    "LGBMRegressor": LGBMRegressor(),
}

In [ ]:
params_dict = {
    "LinearRegression": {},
    # "Ridge": {"alpha": [0.1, 1, 10]},
    # "Lasso": {"alpha": [0.1, 1, 10]},
    # "SVR": {"C": [0.1, 1, 10], "kernel": ["linear", "rbf"]},
    # "DecisionTreeRegressor": {"max_depth": [3, 5, 10]},
    # "RandomForestRegressor": {"n_estimators": [50, 100, 200], "max_depth": [3, 5, 10]},
    # "GradientBoostingRegressor": {"n_estimators": [50, 100, 200], "max_depth": [3, 5, 10]},
    "XGBRegressor": {"n_estimators":[1200],
                     "max_depth":[10],
                     "learning_rate":[0.01],
                     "min_child_weight": [1],
                     "subsample": [0.6],
                     "colsample_bytree": [0.8],
                     "reg_lambda": [0.3]
        },
    "LGBMRegressor": {"n_estimators":[1200],
                      "max_depth":[10],
                      "learning_rate":[0.01]
    }
}

In [ ]:
from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import GridSearchCV

In [ ]:
best_model_dict = {}
for model_name, model in grid_dict.items():
  grid_search = GridSearchCV(estimator=model, param_grid=params_dict[model_name])
  grid_search.fit(X_train, y_train)
  best_model_dict[model_name] = grid_search.best_estimator_
  print(f"{model_name}: {grid_search.best_params_}")
  print(f"Train score: {grid_search.score(X_train, y_train)}")
  print(f"Test score: {grid_search.score(X_test, y_test)}")
  print(f"Train mae: {mae(y_train, grid_search.predict(X_train))}")
  print(f"Test mae: {mae(y_test, grid_search.predict(X_test))}")
  print()

LinearRegression: {}
Train score: 0.36621273560363254
Test score: 0.3117677874309208
Train mae: 7252.724222759578
Test mae: 7441.089803138379

XGBRegressor: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 10, 'min_child_weight': 1, 'n_estimators': 1200, 'reg_lambda': 0.3, 'subsample': 0.6}
Train score: 0.9333192706108093
Test score: 0.7406768798828125
Train mae: 2063.765380859375
Test mae: 3672.588134765625

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002081 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 492
[LightGBM] [Info] Number of data points in the train set: 11110, number of used features: 63
[LightGBM] [Info] Start training from score 14579.510711
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[Ligh

In [ ]:
# # muzeme ulozit natrenovany model, pripadne o nacist
import pickle
# pickle.dump(best_model_dict["XGBRegressor"], open("xgb_final_regressor.pkl", "wb"))
model_xgb = pickle.load(open("xgb_final_regressor.pkl", "rb"))

In [ ]:
test_pred = model_xgb.predict(X_test)

In [ ]:
# modelova diagnosika - graf skutecnych hodnot oproti predikcim

from plotly import express as px
fig = px.scatter(y=y_test, x=test_pred, title="Predicted vs Actual hodnoty", trendline="ols", trendline_color_override="red", width=800, height=800)
fig.update_layout(xaxis_title="Predicted", yaxis_title="Actual")
fig.update_xaxes(scaleanchor="x", scaleratio=1)
fig.update_yaxes(scaleanchor="y", scaleratio=1)
fig.show()